# Multi-objective Optimization with PARETO

The purpose of this Jupyter notebook is to show how to apply the epsilon constraint method for multi-objective optimization (MOO) to a PARETO model. We will explore the tradeoff between the competing objectives of minimizing the cost of a produced water network and the subsurface risk incurred by operating the network.

We begin by importing all necessary modules and loading the required data (for this demonstration, we will make use of PARETO's strategic toy case study):

In [ ]:
#####################################################################################################
# PARETO was produced under the DOE Produced Water Application for Beneficial Reuse Environmental
# Impact and Treatment Optimization (PARETO), and is copyright (c) 2021-2024 by the software owners:
# The Regents of the University of California, through Lawrence Berkeley National Laboratory, et al.
# All rights reserved.
#
# NOTICE. This Software was developed under funding from the U.S. Department of Energy and the U.S.
# Government consequently retains certain rights. As such, the U.S. Government has been granted for
# itself and others acting on its behalf a paid-up, nonexclusive, irrevocable, worldwide license in
# the Software to reproduce, distribute copies to the public, prepare derivative works, and perform
# publicly and display publicly, and to permit others to do so.
#####################################################################################################

from pareto.strategic_water_management.strategic_produced_water_optimization import (
    WaterQuality,
    create_model,
    set_objective,
    Objectives,
    solve_model,
    PipelineCost,
    PipelineCapacity,
    Hydraulics,
    RemovalEfficiencyMethod,
    InfrastructureTiming,
    SubsurfaceRisk,
)
from pareto.utilities.get_data import get_data
from pareto.utilities.results import is_feasible, nostdout
from pyomo.environ import Constraint, Param, value, TerminationCondition, SolverStatus
from importlib import resources

# This emulates what the pyomo command-line tools does
# Tabs in the input Excel spreadsheet
set_list = [
    "ProductionPads",
    "CompletionsPads",
    "SWDSites",
    "ExternalWaterSources",
    "WaterQualityComponents",
    "StorageSites",
    "TreatmentSites",
    "ReuseOptions",
    "NetworkNodes",
    "PipelineDiameters",
    "StorageCapacities",
    "InjectionCapacities",
    "TreatmentCapacities",
    "TreatmentTechnologies",
]
parameter_list = [
    "Units",
    "PNA",
    "CNA",
    "CCA",
    "NNA",
    "NCA",
    "NKA",
    "NRA",
    "NSA",
    "FCA",
    "RCA",
    "RNA",
    "RSA",
    "SCA",
    "SNA",
    "ROA",
    "RKA",
    "SOA",
    "NOA",
    "PCT",
    "PKT",
    "FCT",
    "CST",
    "CCT",
    "CKT",
    "RST",
    "ROT",
    "SOT",
    "RKT",
    "Elevation",
    "CompletionsPadOutsideSystem",
    "DesalinationTechnologies",
    "DesalinationSites",
    "BeneficialReuseCost",
    "BeneficialReuseCredit",
    "TruckingTime",
    "CompletionsDemand",
    "PadRates",
    "FlowbackRates",
    "WellPressure",
    "NodeCapacities",
    "InitialPipelineCapacity",
    "InitialPipelineDiameters",
    "InitialDisposalCapacity",
    "InitialTreatmentCapacity",
    "ReuseMinimum",
    "ReuseCapacity",
    "ExtWaterSourcingAvailability",
    "PadOffloadingCapacity",
    "CompletionsPadStorage",
    "DisposalOperationalCost",
    "TreatmentOperationalCost",
    "ReuseOperationalCost",
    "PipelineOperationalCost",
    "ExternalSourcingCost",
    "TruckingHourlyCost",
    "PipelineDiameterValues",
    "DisposalCapacityIncrements",
    "InitialStorageCapacity",
    "StorageCapacityIncrements",
    "TreatmentCapacityIncrements",
    "TreatmentEfficiency",
    "RemovalEfficiency",
    "DisposalExpansionCost",
    "StorageExpansionCost",
    "TreatmentExpansionCost",
    "PipelineCapexDistanceBased",
    "PipelineCapexCapacityBased",
    "PipelineCapacityIncrements",
    "PipelineExpansionDistance",
    "Hydraulics",
    "Economics",
    "ExternalWaterQuality",
    "PadWaterQuality",
    "StorageInitialWaterQuality",
    "PadStorageInitialWaterQuality",
    "DisposalOperatingCapacity",
    "TreatmentExpansionLeadTime",
    "DisposalExpansionLeadTime",
    "StorageExpansionLeadTime",
    "PipelineExpansionLeadTime_Dist",
    "PipelineExpansionLeadTime_Capac",
    "SWDDeep",
    "SWDAveragePressure",
    "SWDProxPAWell",
    "SWDProxInactiveWell",
    "SWDProxEQ",
    "SWDProxFault",
    "SWDProxHpOrLpWell",
    "SWDRiskFactors",
]

with resources.path(
    "pareto.case_studies",
    "strategic_toy_case_study.xlsx",
) as fpath:
    [df_sets, df_parameters] = get_data(fpath, set_list, parameter_list)

## Epsilon constraint method

The first MOO algorithm we will implement is known as the epsilon constraint method. We begin by creating the PARETO model:

In [ ]:
# Model settings
default = {
    "objective": Objectives.cost,
    "pipeline_cost": PipelineCost.distance_based,
    "pipeline_capacity": PipelineCapacity.input,
    "hydraulics": Hydraulics.false,
    "node_capacity": True,
    "water_quality": WaterQuality.false,
    "removal_efficiency_method": RemovalEfficiencyMethod.concentration_based,
    "infrastructure_timing": InfrastructureTiming.false,
    "subsurface_risk": SubsurfaceRisk.exclude_over_and_under_pressured_wells,
}

# Create model
model = create_model(
    df_sets,
    df_parameters,
    default=default,
)

Next, we set the solver options, solve the model, and check that the solution is feasible and optimal.

In [ ]:
options = {
    "deactivate_slacks": True,
    "scale_model": False,
    "scaling_factor": 1000,
    "running_time": 200,
    "gap": 0,
}

results_min_cost = solve_model(model=model, options=options)

assert results_min_cost.solver.termination_condition == TerminationCondition.optimal
assert results_min_cost.solver.status == SolverStatus.ok

with nostdout():
    feasibility_status = is_feasible(model)

assert feasibility_status

We investigate the value of the objective function (minimum cost) and the corresponding subsurface risk:

In [ ]:
min_cost_obj = model.v_Z.value
min_cost_risk = model.v_Z_SubsurfaceRisk.value

print("Minimum cost model")
print("------------------")
print(f"Minimum cost achievable: {min_cost_obj} {model.v_Z.get_units()}")
print(
    f"Risk incurred with minimum cost: {min_cost_risk} {model.v_Z_SubsurfaceRisk.get_units()}"
)

It's straightforward to alter the model to minimize subsurface risk instead of cost, and then reoptimize:

In [ ]:
# Change the objective function in the model from minimum cost to minimum subsurface risk
set_objective(model, Objectives.subsurface_risk)

results_min_risk = solve_model(model=model, options=options)

assert results_min_risk.solver.termination_condition == TerminationCondition.optimal
assert results_min_risk.solver.status == SolverStatus.ok

with nostdout():
    feasibility_status = is_feasible(model)

assert feasibility_status

We investigate the value of the objective function (minimum subsurface risk) and the corresponding cost:

In [ ]:
min_risk_obj = model.v_Z_SubsurfaceRisk.value
min_risk_cost = model.v_Z.value

print("\nMinimum risk model")
print("------------------")
print(f"Minimum risk achievable: {min_risk_obj} {model.v_Z_SubsurfaceRisk.get_units()}")
print(f"Cost incurred with minimum risk: {min_risk_cost} {model.v_Z.get_units()}")

We see that when minimizing subsurface risk, it is possible to achieve an objective function value of zero, but it comes at a prohibitively high cost. Note, however, that it is possible to achieve zero total risk for a much lower cost. The reason the cost reported above is so high is that the optimizer arbitrarily chooses to build many pieces of infrastructure which are not needed, since there is no penalty for doing so.

Note that is not necessarily feasible to achieve zero overall risk for every produced water network model, but it happens to be feasible with this particular model. 

Next, we implement the epsilon constraint method (and in so doing, demonstrate that it is possible to achieve zero risk with a much lower cost than above). We change the objective function back to minimum cost, and then the epsilon constraint method amounts to solving the optimization problem for different fixed values of subsurface risk: 

In [ ]:
# Change objective function back to minimum cost
set_objective(model, Objectives.cost)

# Create lists for results
fixed_risk_vals = list(range(0, 400, 20))  # Subsurface risk values to iterate over
cost_vals_fixed_risk = list()  # List to store corresponding cost values

for r in fixed_risk_vals:
    print("*" * 60)
    print(f"Solving minimum cost model constraining subsurface risk to {r}")
    print("*" * 60)
    model.v_Z_SubsurfaceRisk.fix(r)  # Fix the subsurface risk to the specified value
    results_min_cost = solve_model(model=model, options=options)

    assert results_min_cost.solver.termination_condition == TerminationCondition.optimal
    assert results_min_cost.solver.status == SolverStatus.ok

    with nostdout():
        feasibility_status = is_feasible(model)

    assert feasibility_status

    cost_vals_fixed_risk.append(model.v_Z.value)

fixed_risk_vals.append(min_cost_risk)
cost_vals_fixed_risk.append(min_cost_obj)

min_cost_for_zero_risk = cost_vals_fixed_risk[0]

In multi-objective optimization, a solution is said to be [Pareto optimal](https://en.wikipedia.org/wiki/Pareto_efficiency) if an improvement in one objective necessitates a deterioration in another objective. This concept inspired the name for Project PARETO. We can plot the results from the epsilon constraint method as a Pareto frontier:

In [ ]:
# Create plot of Pareto curve
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
scatter = ax.scatter(cost_vals_fixed_risk, fixed_risk_vals, c="blue")
ax.set_xlabel(f"Cost [{model.v_Z.get_units()}]")
ax.set_ylabel(f"Subsurface risk [{model.v_Z_SubsurfaceRisk.get_units()}]")

print(f"Cost values [{model.v_Z.get_units()}]: {cost_vals_fixed_risk}")
print(
    f"Subsurface risk values [{model.v_Z_SubsurfaceRisk.get_units()}]: {fixed_risk_vals}"
)

The Pareto curve shows that compared to the pure minimum cost solution, we can significantly reduce the subsurface risk with nearly negligible increases in the cost.

The epsilon constraint method can also be used by minimizing risk for fixed values of cost:

In [ ]:
# Change objective function back to minimum subsurface risk
set_objective(model, Objectives.subsurface_risk)

# Unfix subsurface risk variable
model.v_Z_SubsurfaceRisk.unfix()

# Create lists for results
fixed_cost_vals = list(range(6200, 7800, 100))  # Cost risk values to iterate over
risk_vals_fixed_cost = list()  # List to store corresponding subsurface risk values

for c in fixed_cost_vals:
    print("*" * 60)
    print(f"Solving minimum subsurface risk constraining cost to {c}")
    print("*" * 60)
    model.v_Z.fix(c)  # Fix the cost to the specified value
    results_min_risk = solve_model(model=model, options=options)

    assert results_min_risk.solver.termination_condition == TerminationCondition.optimal
    assert results_min_risk.solver.status == SolverStatus.ok

    with nostdout():
        feasibility_status = is_feasible(model)

    assert feasibility_status

    risk_vals_fixed_cost.append(model.v_Z_SubsurfaceRisk.value)

fixed_cost_vals.append(min_cost_obj)
risk_vals_fixed_cost.append(min_cost_risk)
fixed_cost_vals.append(min_cost_for_zero_risk)
risk_vals_fixed_cost.append(0)

Here is the corresponding plot:

In [ ]:
fig, ax = plt.subplots()
scatter = ax.scatter(fixed_cost_vals, risk_vals_fixed_cost, c="red")
ax.set_xlabel(f"Cost [{model.v_Z.get_units()}]")
ax.set_ylabel(f"Subsurface risk [{model.v_Z_SubsurfaceRisk.get_units()}]")

print(f"Cost values [{model.v_Z.get_units()}]: {fixed_cost_vals}")
print(
    f"Subsurface risk values [{model.v_Z_SubsurfaceRisk.get_units()}]: {risk_vals_fixed_cost}"
)

We can create one plot with both sets of results:

In [ ]:
fig, ax = plt.subplots()
scatter = ax.scatter(
    cost_vals_fixed_risk,
    fixed_risk_vals,
    c="blue",
    label="Fix subsurface risk, minimize cost",
)
scatter = ax.scatter(
    fixed_cost_vals,
    risk_vals_fixed_cost,
    c="red",
    label="Fix cost, minimize subsurface risk",
)
ax.set_xlabel(f"Cost [{model.v_Z.get_units()}]")
ax.set_ylabel(f"Subsurface risk [{model.v_Z_SubsurfaceRisk.get_units()}]")
ax.legend()